In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 4 - M. Massot 2022-2023 - Ecole polytechnique
#    ----------   
#    Exemples préliminaires
#    
#    Auteurs : L. Séries et M. Massot - (C) 2022
#    

# Exemples préliminaires

In [ ]:
import numpy as np
from mpmath import mp
import mpmath

## Matrice de Hilbert

Une matrice de Hilbert est une matrice carrée de terme général :

$$ H_{ij} = \frac{1}{i+j-1} $$

La matrice de Hilbert de taille 6 s'écrit :

$$\begin{pmatrix}
 1   & 1/2 & 1/3 & 1/4 & 1/5  & 1/6  \\
 1/2 & 1/3 & 1/4 & 1/5 & 1/6  & 1/7  \\
 1/3 & 1/4 & 1/5 & 1/6 & 1/7  & 1/8  \\
 1/4 & 1/5 & 1/6 & 1/7 & 1/8  & 1/9  \\
 1/5 & 1/6 & 1/7 & 1/8 & 1/9  & 1/10 \\
 1/6 & 1/7 & 1/8 & 1/9 & 1/10 & 1/11  
\end{pmatrix}$$

## Résolution à précision arbitraire

Dans un premier temps, on calcule la solution exacte pour un second membre $b$ dont chaque  composante vaut 1 pour $n = 4$ :

In [ ]:
n = 6
A = matrix(QQ, [[1/(i+j-1) for j in [1..n]] for i in [1..n]])
print("A = ")
print(A.str(unicode=True))

b = vector(QQ, [1 for i in range(0,n)])
print("\nb = ")
print(b.column().str(unicode=True))

x = A\b
print("\nx = ")
print(x.column().str(unicode=True))

Puis, on résoud le même système pour différents formats de nombre à virgule flottante :

In [ ]:
print("Solution exacte  :")
print(x.column().str(unicode=True))

# double precision
mp.prec = 53
A = mpmath.matrix([[1/(i+j-1) for j in [1..n]] for i in [1..n]])
b = mpmath.matrix([1 for i in range(0,n)])
x = mpmath.lu_solve(A, b)
print(f"\nSolution pour une précision de {mp.dps} chiffres significatifs (double précison) : ")
print(x)

# simple precision
mp.prec = 24
A = mpmath.matrix([[1/(i+j-1) for j in [1..n]] for i in [1..n]])
b = mpmath.matrix([1 for i in range(0,n)])
x = mpmath.lu_solve(A, b)
print(f"\nSolution pour une précision de {mp.dps} chiffres significatifs (simple précison) : ")
print(x)

# 3 chiffres significatifs
mp.dps = 4
A = mpmath.matrix([[1/(i+j-1) for j in [1..n]] for i in [1..n]])
b = mpmath.matrix([1 for i in range(0,n)])
x = mpmath.lu_solve(A, b)
print(f"\nSolution pour une précision de {mp.dps} chiffres significatifs : ")
print(x)

## Perturbation du second membre pour une résolution exacte

On initialise une matrice de Hilbert de taille $n$ avec des fractions fractionnelles de sorte de faire calculs exacts.

In [ ]:
n = 6
A = matrix(QQ, [[1/(i+j-1) for j in [1..n]] for i in [1..n]])
print(A.str(unicode=True))

On calcule la solution exacte pour un second membre $b$ dont chaque composante vaut 1

In [ ]:
b = vector(QQ, [1 for i in range(0,n)])
x = A\b
print(x.column().str(unicode=True))

On perturbe la dernière composante du second membre de (1/1000000), on résoud le système et on s'attend à trouver une solution proche du vecteur dont les composantes valents $1$ :

In [ ]:
b[n-1] *= (1+1/(1000000)) #  perturbation du second membre
x_pert = A\b
print("Solution perturbée :")
print(x_pert.column().str(unicode=True))

In [ ]:
print("Ecart entre la solution et la solution perturbée :" )
for i in range(n):
    print(abs(float(x[i]-x_pert[i])))

err = max(abs(float(x[i]-x_pert[i])) for i in range(0,n))
print("\nNorme infinie de l'de l'écart entre la solution et la solution perturbée  :", err)

## Exemple de Forsythe

On considère le système :

$$
\begin{pmatrix}
 0.0001   & 1 \\
 1   & 1 
\end{pmatrix}
\begin{pmatrix}
 x_1  \\
 x_2  
\end{pmatrix}
=
\begin{pmatrix}
 1.0001  \\
 2  
\end{pmatrix}
$$

pour lequel la solution exacte est $x_1 = 1$ et $x_2 = 1$.

On utilise l'algorithme de l’élimination de Gauss pour des flottants avec 3 chiffres significatifs (en base 10).

In [ ]:
def gaussian_elimination_without_pivoting(a, b): 
    n = b.size
    for i in range(n-1):
        # elimination
        li = a[i+1:,i]/a[i,i]
        b[i+1:] = b[i+1:] - li * b[i]
        a[i+1:] = a[i+1:] - li.reshape(n-i-1,1)*a[i]
        
def backward_substitution(a, b):
    n = b.size
    x = b[0]*np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (b[i] - np.sum(a[i,i+1:]*x[i+1:])) / a[i,i]
    return x

def gauss_solve_without_pivoting(a, b):
    ag = np.copy(a) 
    bg = np.copy(b)
    gaussian_elimination_without_pivoting(ag, bg)
    return backward_substitution(ag, bg)

In [ ]:
# precision arbitraire de 3 chiffres significatifs
mp.dps = 3
mp.pretty = True
A = mpmath.matrix([[1e-4, 1], [1, 1]])
b = mpmath.matrix([1.0001, 2])

A = np.array(A.tolist())
b = np.array(b.tolist()).reshape(2)
x = gauss_solve_without_pivoting(A, b)
print(f"Solution pour une précision de {mp.dps} chiffres significatifs : ")
print(f"x1 = {x[0]}  et x2 = {x[1]}")

On intervertit les 2 lignes du système : 

$$
\begin{pmatrix}
 1   & 1 \\
 0.0001  & 1 
\end{pmatrix}
\begin{pmatrix}
 x_2  \\
 x_1  
\end{pmatrix}
=
\begin{pmatrix}
 2 \\ 
 1.0001 
\end{pmatrix}
$$

In [ ]:
# precision arbitraire de 3 chiffres significatifs
mp.dps = 3
mp.pretty = True
A = mpmath.matrix([[1, 1], [1e-4, 1]])
b = mpmath.matrix([2, 1.0001])

A = np.array(A.tolist())
b = np.array(b.tolist()).reshape(2)
x = gauss_solve_without_pivoting(A, b)
print(f"Solution pour une précision de {mp.dps} chiffres significatifs : ")
print(f"x1 = {x[1]}  et x2 = {x[0]}")